# **Adatok kijelzése grafikonon**

### Importálások

In [ ]:
import numpy as np
import pandas as pd
import pandas_datareader as web
import matplotlib.pyplot as plt
import sys
import plotly.graph_objects as go
import plotly.express as px
import datetime
from sklearn.svm import SVR
from sklearn.neighbors import KernelDensity
plt.style.use('seaborn-darkgrid')

### Beolvassuk a CSV-t

In [ ]:
df_raw = web.DataReader('NVDA', data_source='yahoo', start='2012-01-01', end='2020-01-01')
df_raw.head()

In [ ]:
df_raw = df_raw.reset_index()
df_raw.head()



In [ ]:
df_raw.iat[1,0]
df_raw.info()


### Egy táblázatba hozzuk az adatokat. Open - Max - Min - Close.

##### Betöltjük egy táblába a dátumot és az adatokat egymás után

In [ ]:
df_Date = []
for x in range(0, len(df_raw)):
    for y in range (0,4):
       date_time_str ="0000-00-00 "+str(9+y*4)+":00"
       date_time = df_raw.iat[x,0] + datetime.timedelta(hours=(9+y*4),minutes=0)
       df_Date.append(date_time)
df_Value = []
for x in range(0, len(df_raw)):
    for y in range (1,5):
        df_Value.append(df_raw.iat[x,y])

##### Létrehozzuk az új df-et

In [ ]:
data = {'date':df_Date, 'stock1':df_Value}
df = pd.DataFrame(data)
df.head()

### Adat ellenőrzés

In [ ]:
#df.iloc[1,1]
#df.isnull().sum()
#df.info()
#df.describe()
#df.plot()
#df.tail(50)

### Destabilitásvizsgálat

#### az utolsó x elemen veszi a legnagyobb különbséget és elossza az átlagos függvényértékkel

In [ ]:
#amennyi az a/7 annyi adatra kell illeszteni a magasfrekvenciás polinomot!

x, y, z, a = 0, 0, 0, 0
df_dst = df
a=50
for z in range(0,3):
  dst = []
  a=a+a*z
  for y in range(0,a):
    dst.append(0)
  for x in range(a,df.shape[0]):
    dif = (df[x:x+a].max()-df[x:x+a].min()).array[1]
    dem = (df[x:x+a].mean(axis = 0,numeric_only=True)).array[0]
    dst.append(dif/dem)
  cln = 'Dest'+str(a)
  df_dst[cln] = dst

#df_dst.tail(10)

In [ ]:
fig = px.line(df, x='date', y='stock1')

fig.show()

In [ ]:
fig = px.line(df_dst, x="date", y='Dest50')
fig.show()

Most, hogy elő vannak készítve az adatok napi bontásban..
**Mennyire VAD a függvény**
Megnézem minden időpillanatban "d" értékre a MAX és MIN különbségét és elosztom a piaci értékkel 

Az adatokra valami mozgó átlagot kell illeszteni végig, majd annak venni az első és másidik deriváltjait a vizsgálati idő szerinti mennyiségű adaton, később ezt fogja helyettesíteni az adatokra illesztett polinom

In [ ]:
actual_price = df_dst.tail(1)
#actual_price.head(10)

In [ ]:
df_days = df_dst.loc[:, 'date'].tail(300)
df_prices = df_dst.loc[:,'stock1'].tail(300)

In [ ]:
days=list()
prices=list()
for x in range(0,df_days.shape[0]):
  days.append(int(x))
for price in df_prices:
  prices.append(float(price))

D = np.array(days).reshape(-1, 1)
p = np.array(prices).reshape(-1, 1)
p = p.ravel()

In [ ]:
lin_svr = SVR(kernel='linear', C=1000.0)
lin_svr.fit(D, p)

poly_svr = SVR(kernel='poly', C=1000.0, degree = 3)
poly_svr.fit(D, p)

rbf_svr = SVR(kernel='rbf', C=1000.0, gamma = 0.15)
rbf_svr.fit(D, p)

In [ ]:
plt.figure(figsize=(17,7))
plt.scatter(D, prices, color='black', label='Original Data')
#plt.plot(D, lin_svr.predict(D), color='green', label="RBF")
plt.plot(D, poly_svr.predict(D), color='orange', label="Poly")
#plt.plot(D, rbf_svr.predict(D), color='purple', label="Lin")
plt.legend()
plt.show()

In [ ]:
from matplotlib import pyplot
from numpy.random import normal
from numpy import hstack
from numpy import asarray
from numpy import exp
from sklearn.neighbors import KernelDensity
# generate a sample
sample1 = normal(loc=200, scale=2, size=3000)
sample2 = normal(loc=400, scale=2, size=7000)
sample = hstack((sample1, sample2))
# fit density
model = KernelDensity(bandwidth=5, kernel='gaussian')
sample = p.reshape(-1,1)
model.fit(sample)
# sample probabilities for a range of outcomes
values = D
values = values.reshape((len(values), 1))
probabilities = model.score_samples(values)
probabilities = exp(probabilities)
# plot the histogram and pdf
pyplot.hist(sample, bins=100, density=True)
pyplot.plot(values[:], probabilities)
pyplot.show()
